In [8]:
%load_ext autoreload
%autoreload 2

import json

split = 'val'
results = json.load(open('predictions.json'))
dataset = json.load(open('/data/vqa/annotations/VizWiz/VizWiz_QA/Annotations/%s.json'%split))

img2gt = {x['image']:x['answers'] for x in dataset}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#print(dataset[:10])
#results[:30]
#dataset[18]

In [10]:
print(results[0])
print(dataset[0])

{'image': 'VizWiz_val_00000000.jpg', 'answer': 'no'}
{'image': 'VizWiz_val_00000000.jpg', 'question': 'Ok. There is another picture I hope it is a better one.', 'answers': [{'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'maybe'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'yes'}, {'answer': 'unanswerable', 'answer_confidence': 'no'}, {'answer': 'cannot repair this computer automatically', 'answer_confidence': 'maybe'}, {'answer': 'blank screen', 'answer_confidence': 'yes'}], 'answer_type': 'unanswerable', 'answerable': 0}


In [11]:
print(results[0]['image'])

VizWiz_val_00000000.jpg


In [12]:
imdir='VizWiz_%s_%012d.jpg'
print(dataset[0]['image'])
img = imdir%(split,28000)
print(img)

VizWiz_val_00000000.jpg
VizWiz_val_000000028000.jpg


In [13]:
from collections import Counter
img2ans_type = {}
for one_data in dataset:
    ans_counter = Counter([x['answer'] for x in one_data['answers']])
    ans = ans_counter.most_common(1)[0][0]
    if ans == 'yes' or ans == 'no':
        ans_type = 'yes/no'
    elif ans == 'unanswerable' or ans == 'unsuitable':
        ans_type = 'unanswerable'
    elif ans.isdigit():
        ans_type = 'number'
    else:
        ans_type = 'other'
    img2ans_type[one_data['image']] = ans_type
    
all_ans = list(img2ans_type.values())
for ans_type in set(all_ans):
    print (ans_type, ':', all_ans.count(ans_type)*1.0/len(all_ans))

yes/no : 0.05232692752952072
unanswerable : 0.3202130122713591
other : 0.6158833063209076
number : 0.01157675387821255


In [15]:
import numpy as np
img2acc = {}
imdir='VizWiz_%s_%012d.jpg'

for pred in results:
    #img = imdir%(split,pred['question_id']) #Reconstruct image name from question_id
    img = pred['image']
    pred_ans = pred['answer']
    gt_ans = img2gt[img]
    gt_ans = [x['answer'] for x in gt_ans]
    gt_ans = [x.lower() for x in gt_ans]
    cur_acc = np.minimum(1.0, gt_ans.count(pred_ans)/3.0)
    img2acc[img] = cur_acc

print ('Accuracy :', np.mean(list(img2acc.values())))
for ans_type in set(all_ans):
    acc_per_type = np.mean([acc for img, acc in img2acc.items() if img2ans_type[img] == ans_type])
    print (ans_type, ':', acc_per_type)

Accuracy : 0.42478968897121244
yes/no : 0.5634218289085545
unanswerable : 0.8013979272113764
other : 0.2206766917293233
number : 0.24


In [3]:
"""Denis prepro validation balanced"""
!pip install pycocoevalcap

     |████████████████████████████████| 104.3 MB 102.7 MB/s eta 0:00:01
  Using cached pycocotools-2.0.2.tar.gz (23 kB)
  Using cached Cython-0.29.21-cp36-cp36m-manylinux1_x86_64.whl (2.0 MB)
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=281749 sha256=52ea542d3519fb63b1a3c088445fb09d9080c5bf3b98156c8358dd55d938c9ff
  Stored in directory: /home/cc67459/.cache/pip/wheels/d8/c2/ba/8f5306f921c2e79ad7b09effdfed6bd966cfcf8c6fe55422d6
Successfully built pycocotools


In [17]:
# Download coco-caption from https://github.com/tylin/coco-caption
import sys
sys.path.insert(0, '../coco-caption')

from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

class COCOEvalCap:
    def __init__(self,images,gts,res):
        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.params = {'image_id': images}
        self.gts = gts
        self.res = res

    def evaluate(self):
        imgIds = self.params['image_id']
        gts = self.gts
        res = self.res

        # =================================================
        # Set up scorers
        # =================================================
        print ('tokenization...')
        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)

        # =================================================
        # Set up scorers
        # =================================================
        print ('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr")
        ]

        # =================================================
        # Compute scores
        # =================================================
        eval = {}
        for scorer, method in scorers:
            print ('computing %s score...'%(scorer.method()))
            assert(set(gts.keys()) == set(res.keys()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, imgIds, m)
                    print("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, imgIds, method)
                print ("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

In [34]:
#res = {imdir%(split,x['question_id'])):[{'image_id':imdir%(split,x['question_id']), 'caption':x['answer']}] for x in results}
print(results[0]['image'])
res={}
res = {x['image']:[{'image_id':x['image'], 'caption':x['answer']}] for x in results}

gts = {}
for img, ans_list in img2gt.items():
    ans_list = [x['answer'] for x in ans_list]
    tmp = []
    for x in ans_list:
        try:
            tmp.append(str(x))
        except:
            pass
    ans_list = tmp
    ans_list = [{'image_id': img, 'caption': str(x)} for x in ans_list]
    gts[img] = ans_list
for img in gts.keys():
    if img not in res.keys():
        res[img] = [{'image_id':img, 'caption':''}]

#### CHANGED CODE OF BLEU/METEOR SINCE RAISES ERROR WHEN COMPARING gts.keys() == res.keys()
       
evalObj = COCOEvalCap(gts.keys(),gts,res)
evalObj.evaluate()
print (evalObj.eval)

VizWiz_val_00000000.jpg
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4387, 'reflen': 4649, 'guess': [4387, 68, 18, 7], 'correct': [2537, 22, 3, 1]}
ratio: 0.9436437943641763
Bleu_1: 0.545
Bleu_2: 0.407
Bleu_3: 0.297
Bleu_4: 0.243
computing METEOR score...
METEOR: 0.275
computing Rouge score...
ROUGE_L: 0.570
computing CIDEr score...
CIDEr: 0.531
{'Bleu_1': 0.54477345632856, 'Bleu_2': 0.4074707197201333, 'Bleu_3': 0.2965069581517403, 'Bleu_4': 0.24337021935611858, 'METEOR': 0.27484944364080693, 'ROUGE_L': 0.5696490274404493, 'CIDEr': 0.530886892790464}
